In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import glob
import collections
import numpy as np
from tqdm import tqdm
from PIL import Image

In [ ]:
os.chdir('gdrive/My Drive/hotdog_data')

In [ ]:
random_state=seed

In [4]:
image_list = []
for filename in tqdm(glob.glob('train/hotdog/*.jpg')):
    im = Image.open(filename).convert('RGB')  # in case some are grayscale
    image_list.append((im, 1))

100%|██████████| 708/708 [00:03<00:00, 188.24it/s]


In [5]:
for filename in tqdm(glob.glob('train/not_hotdog/*.jpg')):
    im = Image.open(filename).convert('RGB')
    image_list.append((im, 0))

100%|██████████| 963/963 [00:05<00:00, 179.43it/s]


In [ ]:
image_list = [(item[0].rotate(90, expand=True),item[1]) if item[0].size[0]<item[0].size[1] else item for item in image_list]
image_list = [(item[0].resize((500,375)),item[1]) for item in image_list]

In [ ]:
X_train = [np.array(item[0]) for item in image_list]
y_train = [item[1] for item in image_list]

# Задание 2. Tensorflow 2.0 / Keras.

In [ ]:
y_train = np.array(y_train).reshape(1671,1)

In [ ]:
X_train = np.array(X_train).reshape(1671, 500, 375, 3).astype('float32')

In [ ]:
X_train /= 255

In [ ]:
from sklearn.model_selection import train_test_split

In [12]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
print(tf.__version__)
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping

2.2.0-rc3


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=seed)

In [ ]:
X_test, X_dev, y_test, y_dev = train_test_split(X_test, y_test, test_size=0.5, random_state=seed)  # 0.7 0.15 0.15 train test dev

In [15]:
X_train.shape, y_train.shape

((1169, 500, 375, 3), (1169, 1))

In [ ]:
batch_size = 32
epochs = 12
input_shape = (500,375,3)

In [ ]:
model = Sequential()
model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=input_shape, padding="same"))
model.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model.add(Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))
model.add(Flatten())
model.add(Dense(84, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [18]:
callback = EarlyStopping(monitor='val_loss', patience=2)

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

history=model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks = [callback],
          verbose=1,
          validation_data=(X_dev, y_dev))

Epoch 1/12
37/37 [==============================] - 10s 260ms/step - loss: 5.9000 - accuracy: 0.6749 - val_loss: 0.5555 - val_accuracy: 0.7371
Epoch 2/12
37/37 [==============================] - 9s 236ms/step - loss: 0.4729 - accuracy: 0.7896 - val_loss: 0.4805 - val_accuracy: 0.7649
Epoch 3/12
37/37 [==============================] - 9s 236ms/step - loss: 0.3910 - accuracy: 0.8417 - val_loss: 0.4236 - val_accuracy: 0.8247
Epoch 4/12
37/37 [==============================] - 9s 237ms/step - loss: 0.3714 - accuracy: 0.8426 - val_loss: 0.4436 - val_accuracy: 0.7729
Epoch 5/12
37/37 [==============================] - 9s 236ms/step - loss: 0.2731 - accuracy: 0.8888 - val_loss: 0.4549 - val_accuracy: 0.8008


In [19]:
model.evaluate(X_test, y_test)

8/8 [==============================] - 1s 80ms/step - loss: 0.3997 - accuracy: 0.8446


[0.39965322613716125, 0.8446215391159058]

Даже такая сеть без особого тюнинга уже показывает результаты в 0.8446 (ошибка в 0.1554 соответственно, сравним с ~0.2 для SVC). Плюс обучение произошло гораздо быстрее, чем с SVM. Таким образом, CNN лучше приспособлена для выполнения данной задачи. На очереди VGG!